In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import joblib

%matplotlib inline

# 데이터 전처리

In [40]:
lol_df = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/CL15_fix.csv")

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
def train_test(df):
  x_train = df.drop(['blueWins', 'matchMin'], axis=1)
  y_train = df['blueWins']
  return x_train, y_train

In [41]:
x_train, y_train = train_test(lol_df)

val_size = 0.2

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=val_size, random_state=42)

# 15분 데이터

## LGBM 모델 적용

In [6]:
pip install lightgbm


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import plot_importance, plot_metric, plot_tree

In [8]:
model = LGBMClassifier()
model.fit(x_train, y_train)
valid_pred = model.predict(x_valid)
accuracy = accuracy_score(y_valid, valid_pred)
print("Accuracy: ", accuracy*100)

Accuracy:  77.37704918032787


# 베이지안 optimization

In [9]:
pip install bayesian-optimization


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from bayes_opt import BayesianOptimization

In [11]:
# 탐색 대상 함수 (XGBRegressor)
def LGBM_cv(max_depth,learning_rate, n_estimators,
            min_child_weight, subsample
            ,colsample_bytree, silent=True):

    # 모델 정의
    model = LGBMClassifier(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            min_child_weight=min_child_weight,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree
                            )
    # 모델 훈련
    model.fit(x_train, y_train)

    # 예측값 출력
    y_pred= model.predict(x_valid)

    return f1_score(y_valid, y_pred)

In [19]:
pbounds = {'max_depth': (3, 10),
                'learning_rate': (0.01, 0.3),
                'n_estimators': (20,100),
                'min_child_weight': (0, 3),
                'subsample': (0.5, 1),
                'colsample_bytree' :(0.2, 1)
                }

lm_bo=BayesianOptimization(f=LGBM_cv, pbounds=pbounds, verbose=2, random_state=1 )    

lm_bo.maximize(init_points=2, n_iter=100)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | 0.7612    | 0.5336    | 0.2189    | 3.001     | 0.907     | 31.74     | 0.5462    |
| 2         | 0.7865    | 0.349     | 0.1102    | 5.777     | 1.616     | 53.54     | 0.8426    |
| 3         | 0.763     | 0.5736    | 0.2719    | 6.849     | 2.075     | 53.05     | 0.646     |
| 4         | 0.7368    | 0.6782    | 0.1023    | 7.641     | 1.345     | 33.78     | 0.6515    |
| 5         | 0.7556    | 0.8633    | 0.1507    | 5.612     | 1.403     | 53.87     | 0.7718    |
| 6         | 0.7586    | 0.5276    | 0.04547   | 8.11      | 2.604     | 73.03     | 0.6975    |
| 7         | 0.7433    | 0.9334    | 0.03744   | 9.907     | 2.841     | 21.07     | 0.727     |
| 8         | 0.7566    | 0.7552    | 0.1763    | 5.304     | 2.129     | 83.79     | 0.5152    |
| 9         | 0.7744

In [20]:
print(lm_bo.max['params'])

{'colsample_bytree': 0.23819957644862377, 'learning_rate': 0.16760611872110565, 'max_depth': 3.863764422182043, 'min_child_weight': 1.6153122699704658, 'n_estimators': 84.53631534990768, 'subsample': 0.720688877816275}


In [21]:
param = {'colsample_bytree': 0.23819957644862377, 'learning_rate': 0.16760611872110565, 'max_depth': 4, 'min_child_weight': 1.6153122699704658, 
         'n_estimators': 85, 'subsample': 0.720688877816275}
model = LGBMClassifier(**param)
model.fit(x_train, y_train)
valid_pred = model.predict(x_valid)
accuracy = accuracy_score(y_valid, valid_pred)
print("Accuracy: ", accuracy*100)

Accuracy:  78.68852459016394


DATA 불러오기

In [48]:
br15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/BR15_fix.csv")
di15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/DI15_fix.csv")
gl15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/GL15_fix.csv")
ir15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/IR15_fix.csv")
pl15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/PL15_fix.csv")
si15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/SI15_fix.csv")

data15 = [ir15, si15, gl15, pl15 ,di15]
data_label = ["ir15", "si15", "gl15", "pl15" ,"di15"]

In [49]:
for i,data in enumerate(data15):
    x_train = data.drop(["matchMin","blueWins"], axis=1)
    y_train = data["blueWins"]
    y_pred = model.predict(x_train)
    accuracy = accuracy_score(y_train, y_pred)
    print(data_label[i])
    print("Accuracy: ", accuracy*100)

ir15
Accuracy:  59.5
si15
Accuracy:  60.29411764705882
gl15
Accuracy:  51.243781094527364
pl15
Accuracy:  68.96551724137932
di15
Accuracy:  77.83251231527095


In [51]:
x_train = br15.drop(["blueWins"], axis=1)
y_train = br15["blueWins"]
y_pred = model.predict(x_train)
accuracy = accuracy_score(y_train, y_pred)
print("br15")
print("Accuracy: ", accuracy*100)

br15
Accuracy:  55.72139303482587
